In [1]:
# first, let us create our inputs
import gym 
from nn import PolicyNN, ValueNN

# lets get our environment
env = gym.make("CartPole-v1")

# Policy parameters (NN) = "theta". 2 hidden layers w/ 32 nodes. softmax output for each action
theta = PolicyNN(
    env.observation_space.shape[0],
    2,
    32,
    env.action_space.n
    )

# Value function parameters (NN) = "V". outputs value of state (scalar)
V = ValueNN(
    env.observation_space.shape[0],
    2,
    32,
    1
)